In [26]:
import pandas as pd
import polars as pl
import numpy as np
from temporal_fusion_transformer.src.datasets.electricity import read_raw_csv_v2, read_raw_csv

csv_path = "/Users/artemsereda/Documents/IdeaProjects/temporal_fusion_transformer/data/electricity/LD2011_2014.txt"

In [27]:
read_raw_csv(csv_path)

  0%|          | 0/35065 [00:00<?, ?it/s]

,power_usage,hours_from_start,days_from_start,id,hour,day_of_week,month
17544,2.538071,26304.0,1096,MT_001,0,2,1
17545,2.855330,26305.0,1096,MT_001,1,2,1
17546,2.855330,26306.0,1096,MT_001,2,2,1
17547,2.855330,26307.0,1096,MT_001,3,2,1
17548,2.538071,26308.0,1096,MT_001,4,2,1
...,...,...,...,...,...,...,...
10461482,20824.324324,32299.0,1345,MT_370,19,6,9
10461483,19527.027027,32300.0,1345,MT_370,20,6,9
10461484,20202.702703,32301.0,1345,MT_370,21,6,9
10461485,19851.351351,32302.0,1345,MT_370,22,6,9


In [28]:
read_raw_csv_v2(csv_path)

  0%|          | 0/370 [00:00<?, ?it/s]

timestamp,power_usage,hours_from_start,days_from_start,hour,day,day_of_week,month,id
datetime[ms],f64,f64,i64,u32,u32,u32,u32,str
2014-01-01 00:00:00,2.538071,26304.0,1096,0,1,3,1,"""MT_001"""
2014-01-01 01:00:00,2.85533,26305.0,1096,1,1,3,1,"""MT_001"""
2014-01-01 02:00:00,2.85533,26306.0,1096,2,1,3,1,"""MT_001"""
2014-01-01 03:00:00,2.85533,26307.0,1096,3,1,3,1,"""MT_001"""
2014-01-01 04:00:00,2.538071,26308.0,1096,4,1,3,1,"""MT_001"""
2014-01-01 05:00:00,2.538071,26309.0,1096,5,1,3,1,"""MT_001"""
2014-01-01 06:00:00,3.172589,26310.0,1096,6,1,3,1,"""MT_001"""
2014-01-01 07:00:00,2.85533,26311.0,1096,7,1,3,1,"""MT_001"""
2014-01-01 08:00:00,1.269036,26312.0,1096,8,1,3,1,"""MT_001"""
